In [1]:
import mlflow
import pandas as pd 
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import nltk
from nltk.stem import WordNetLemmatizer
from revised_stopwords import get_revised_stopwords
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msurahman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\msurahman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
df= pd.read_csv('IMDB_Dataset.csv')
df=df.sample(5000)
df.to_csv('data.csv',index=False)


In [3]:
# x = df['sentiment'].isin(['positive','negative'])
# df = df[x]
# check if there two values or it contains any other too 

In [4]:

x=df["review"]
df['sentiment']=df["sentiment"].replace({
  'positive' or 'Positive':1,
  'negative' or 'negative':0
  
})

C:\Users\msurahman\AppData\Local\Temp\ipykernel_15224\2316282240.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment']=df["sentiment"].replace({


In [5]:
y= df["sentiment"]

In [6]:
import string
nltk.download('wordnet')
def normalization(text):
    # 1. Lowercase and URL removal (Best to do while it's still one string)
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
  
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 3. Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # 4. Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 5. Tokenize, Stopword removal, and Lemmatization
    lemmatizer = WordNetLemmatizer()
    stop_words = set(get_revised_stopwords()) 
    words = text.split()
    cleaned_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    
    return " ".join(cleaned_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\msurahman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
x=[normalization(text) for text in x]
preprocessed_df=pd.DataFrame({"review":x,"sentiment":y})
preprocessed_df.head(4)

,review,sentiment
3005,pick year event basic instinct catherine trame...,1
7335,wonderful film never failed move colour convin...,1
43334,disastermovie meaning word every character eve...,0
17703,would promising student film never seems know ...,0


In [8]:
preprocessed_df.to_csv('preprocessed_data.csv',index=False)



In [9]:
df=pd.read_csv("preprocessed_data.csv")
df

,review,sentiment
0,pick year event basic instinct catherine trame...,1
1,wonderful film never failed move colour convin...,1
2,disastermovie meaning word every character eve...,0
3,would promising student film never seems know ...,0
4,tuned thing one night cable channel minute cre...,0
...,...,...
4995,inherent problem staging merchant venice never...,1
4996,movie released biggest hit soon became blockbu...,0
4997,meanspirited ugly nasty retroaction thriller b...,1
4998,well goethe said really isnt point trying pas ...,0


In [10]:
df["sentiment"].value_counts()

sentiment
0    2520
1    2480
Name: count, dtype: int64

In [11]:
df.isnull().sum()


review       0
sentiment    0
dtype: int64

In [12]:
vectorizer=CountVectorizer(max_features=1000)
X=vectorizer.fit_transform(df['review'])
y=df['sentiment']


In [13]:
print(X[10])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 37 stored elements and shape (1, 1000)>
  Coords	Values
  (0, 133)	1
  (0, 600)	1
  (0, 502)	2
  (0, 637)	1
  (0, 954)	1
  (0, 329)	4
  (0, 948)	1
  (0, 145)	1
  (0, 983)	1
  (0, 241)	1
  (0, 873)	1
  (0, 364)	1
  (0, 515)	1
  (0, 323)	2
  (0, 969)	1
  (0, 91)	1
  (0, 423)	1
  (0, 660)	1
  (0, 369)	1
  (0, 475)	1
  (0, 737)	1
  (0, 103)	1
  (0, 786)	1
  (0, 789)	1
  (0, 497)	1
  (0, 25)	1
  (0, 622)	1
  (0, 433)	1
  (0, 610)	1
  (0, 253)	1
  (0, 621)	1
  (0, 495)	1
  (0, 399)	1
  (0, 275)	1
  (0, 343)	1
  (0, 606)	1
  (0, 538)	1


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/MSURahmands365/end_to_end_mlops_pr.mlflow')
dagshub.init(repo_owner='MSURahmands365', repo_name='end_to_end_mlops_pr', mlflow=True)
mlflow.set_experiment("Logistic Regression Base Line")


Accessing as MSURahmands365

Initialized MLflow to track repo "MSURahmands365/end_to_end_mlops_pr"

Repository MSURahmands365/end_to_end_mlops_pr initialized!

<Experiment: artifact_location='mlflow-artifacts:/b55325c816e94b0ab7eb1a5b0ff7ff1e', creation_time=1769431400143, experiment_id='0', last_update_time=1769431400143, lifecycle_stage='active', name='Logistic Regression Base Line', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 1000)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2026-01-27 12:46:25,750 - INFO - Starting MLflow run...
2026-01-27 12:46:26,615 - INFO - Logging preprocessing parameters...
2026-01-27 12:46:27,733 - INFO - Initializing Logistic Regression model...
2026-01-27 12:46:27,735 - INFO - Fitting the model...
2026-01-27 12:46:27,917 - INFO - Model training complete.
2026-01-27 12:46:27,918 - INFO - Logging model parameters...
2026-01-27 12:46:28,364 - INFO - Making predictions...
2026-01-27 12:46:28,366 - INFO - Calculating evaluation metrics...
2026-01-27 12:46:28,384 - INFO - Logging evaluation metrics...
2026-01-27 12:46:29,909 - INFO - Saving and logging the model...
2026/01/27 12:46:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/27 12:46:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026-01-27 12:46:46,279 - INFO - Model training and logging completed in 19.66 seconds.
2026-

🏃 View run magnificent-ray-187 at: https://dagshub.com/MSURahmands365/end_to_end_mlops_pr.mlflow/#/experiments/0/runs/47fe4da654914dd7bf5c67fac23514f8
🧪 View experiment at: https://dagshub.com/MSURahmands365/end_to_end_mlops_pr.mlflow/#/experiments/0
